In [ ]:
using Pkg; Pkg.activate("cg")

In [ ]:
using LinearAlgebra, Printf

In [ ]:
function cg(A, b; atol=1.0e-8, rtol=1.0e-6)
    # Resout le systeme Ax=b, i.e., min -b'x + 1/2 x'Ax.
    # A doit etre symmetrique et definie positive.
    n = length(b)
    x = zeros(n)
    r = -b
    rNorm = norm(r)
    rNorm0 = rNorm
    rNorms = [rNorm]
    p = -r
    k = 0
    @printf("%3d  %7.1e\n", k, rNorm)
    while (k < 2 * n) && (rNorm > atol + rtol * rNorm0)
        Ap = A * p
        pAp = dot(p, Ap)
        pAp > 0 || (@warn "A n'est pas definie positive !")
        α = rNorm^2 / pAp
        x = x + α * p
        r = r + α * Ap
        rNorm_new = norm(r)
        β = (rNorm_new / rNorm)^2
        rNorm = rNorm_new
        push!(rNorms, rNorm)
        p = -r + β * p
        k = k + 1
        @printf("%3d  %7.1e\n", k, rNorm)
    end
    return x, rNorms
end

In [ ]:
A = rand(5, 5) ; A = A * A'; b = rand(5);

In [ ]:
cg(A, b);

In [ ]:
eye(n) = Matrix(1.0*I, n, n)
cg(eye(5), b);

In [ ]:
A = rand(50, 50); A = A * A'; b = A * ones(50);
x, rNorms = cg(-A, -b);

In [ ]:
using Plots
plot(rNorms, yscale=:log10, linewidth=2, label="resid norm")

In [ ]:
A = [1.0 0 ; 0 0]; b = [1.0 ; 0.0]; cg(A, b);

In [ ]:
(Q, R) = qr(rand(50, 50));

In [ ]:
d = [ones(10) ; 4 * ones(10) ; 10 * ones(20) ; 2 * ones(10)];

In [ ]:
A = Q * diagm(0 => d) * Q';

In [ ]:
size(A)

In [ ]:
cg(A, rand(50));

In [ ]:
A = rand(10, 4); b = rand(10);

In [ ]:
cg(A'*A, A'*b)

In [ ]:
A \ b

In [ ]:
(Q, R) = qr(A)

In [ ]:
A \ b

In [ ]:
using HarwellRutherfordBoeing
HB = HarwellBoeingMatrix("/problems/animal/hb/small2.hb")
A = HB.matrix;
b = HB.rhs[:,1];
size(A)

In [ ]:
"""Scale the columns of a sparse matrix A by their Euclidean norm.
"""
function scale_ls!(A)
  s = ones(size(A, 2))
  for j = 1 : size(A, 2)
    i = A.colptr[j]
    k = A.colptr[j+1] - 1
    nj = i <= k ? norm(A.nzval[i:k]) : 0.0
    if nj > 0.0
      A.nzval[i:k] ./= nj
      s[j] = nj
    end
  end
  return s
end

In [ ]:
s = scale_ls!(A);  # equivaut à une mise à l'échelle des inconnues

In [ ]:
spy(A)

In [ ]:
using Krylov

In [ ]:
x_cgls, stats_cgls = cgls(A, b, atol=1.0e-10, rtol=1.0e-10);
x_lsqr, stats_lsqr = lsqr(A, b, atol=1.0e-10, rtol=1.0e-10, axtol=0.0, btol=0.0, etol=0.0);
x_cg, rNorms_cg = cg(A'*A, A'*b, atol=1.0e-10, rtol=1.0e-10);

In [ ]:
plot(stats_cgls.Aresiduals, yscale=:log10, linewidth=2, alpha=.75, label="resid norm CGLS")
plot!(stats_lsqr.Aresiduals, yscale=:log10, linewidth=2, alpha=.75, label="resid norm LSQR")
plot!(rNorms_cg, yscale=:log10, linewidth=2, alpha=.75, label="resid norm CG")

In [ ]:
norm(x_cgls - x_lsqr) / norm(x_cgls)

In [ ]:
@time cgls(A, b, atol=1.0e-10, rtol=1.0e-10);

In [ ]:
@time lsqr(A, b, atol=1.0e-10, rtol=1.0e-10, axtol=0.0, btol=0.0, etol=0.0);

In [ ]:
@time cg(A'*A, A'*b, atol=1.0e-10, rtol=1.0e-10);